In [1]:
import networkx as nx

import qiskit
from qiskit import Aer, IBMQ, execute
from qiskit.providers.aer import noise
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from qiskit import IBMQ, execute
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
import numpy as np
from qiskit.providers.aer.noise.errors import standard_errors as SE
from qiskit.providers.aer.noise.device import models 
from qiskit import wrapper as w
from qiskit import compile
IBMQ.enable_account('a19585690b1c88c398330f76f4f88f2af27dff7b9ded2dec6c3a681fb3b4a5bdfe7a697952254e4c53c3ff59027d0bfb215f6d340a57fd1584f100fbf4e6074b')
print("Available backends:")
IBMQ.backends()


ImportError: No module named qiskit

In [ ]:
def get_PST(counts_noisy,counts_ideal):
    
    correct_total_ocurance_noisy=0
    correct_total_ocurance_ideal=0
    
    for key, value in counts_ideal.items():
        correct_total_ocurance_ideal += value 
        correct_total_ocurance_noisy += counts_noisy[key]
    PST = correct_total_ocurance_noisy/correct_total_ocurance_ideal
    return PST

In [ ]:
import json
device = IBMQ.get_backend('ibmq_16_melbourne')
properties = device.properties()
Config = json.load(open("../Noise Simulation/Config.txt")) ##### Use this Noise Config
properties=properties.from_dict(Config)

In [ ]:
device = IBMQ.get_backend('ibmq_16_melbourne')
coupling_map = device.configuration().coupling_map

gate_times = [
    ('u1', None, 0), ('u2', None, 100), ('u3', None, 200),
    ('cx', [1, 0], 678), ('cx', [1, 2], 547), ('cx', [2, 3], 721),
    ('cx', [4, 3], 733), ('cx', [4, 10], 721), ('cx', [5, 4], 800),
    ('cx', [5, 6], 800), ('cx', [5, 9], 895), ('cx', [6, 8], 895),
    ('cx', [7, 8], 640), ('cx', [9, 8], 895), ('cx', [9, 10], 800),
    ('cx', [11, 10], 721), ('cx', [11, 3], 634), ('cx', [12, 2], 773),
    ('cx', [13, 1], 2286), ('cx', [13, 12], 1504), ('cx', [], 800)
]

# Construct the noise model from backend properties
# and custom gate times
noise_model = noise.device.basic_device_noise_model(properties, gate_times=gate_times)

In [ ]:
# Read the input QASM circuit
def read_qasm(filepath, verbo):
    #TODO: check if the syntax works 
    circ = QuantumCircuit.from_qasm_file(filepath)
    list_ops = circ.count_ops()
    flag = list_ops.get("measure")
    # add measurement instructions at the end
        
    if flag is None:
        circ.measure(circ.qregs[0], circ.cregs[0])
        if verbo == 1:
            print("measurement instructions not included!!! -- adding measurement")

    if verbo == 1:
        print('Total number of ops: ', circ.size())
        print('circuit depth: ', circ.depth())
        print('number of qubits:', circ.width())
        print('Ops:', circ.count_ops())
    
    return circ

In [ ]:
ghz8 = 'GHZ8_sabre.qasm'
qaoa = 'QAOA1_sabre.qasm'
bv9 = 'bv9_sabre.qasm'
decode = 'decod24_v2_43_sabre.qasm'
fredkin = 'fredkin_sabre.qasm'
ising = 'ising_model_10_sabre.qasm'
qft = 'qft_10_sabre.qasm'

In [ ]:
# Run an ideal simulation
def ideal_simulation(filepath):
    circ_ideal = read_qasm(filepath,0)
    backend = Aer.get_backend('qasm_simulator')
    qobj_ideal=compile(circ_ideal,shots=8192, backend=backend)
    exp_out = backend.run(qobj_ideal)
    counts_ideal = exp_out.result().get_counts()
    #print('Counts on ideal simulator')
    #print(counts_ideal)
    return counts_ideal

In [ ]:
def run_simulation(workload):
    if(workload==1):
        circ = read_qasm(bv9,0)
    elif(workload==2):
        circ = read_qasm(ghz8,0) 
    elif(workload==3):
        circ = read_qasm(fredkin,0) 
    elif(workload==4):
        circ = read_qasm(decode,0)   
    elif(workload==5):
        circ = read_qasm(qaoa,0)        
    basis_gates = noise_model.basis_gates
    # Select the QasmSimulator from the Aer provider
    simulator = Aer.get_backend('qasm_simulator')
    if(workload == 1):
        initial_layout = {("q", 0): ("q", 3), 
                          ("q", 1): ("q", 2), 
                          ("q", 2): ("q", 10),
                          ("q", 3): ("q", 11), 
                          ("q", 4): ("q", 5),
                          ("q", 5): ("q", 9),
                          ("q", 6): ("q", 8),
                          ("q", 7): ("q", 6),
                          ("q", 8): ("q", 7),
                          ("q", 9): ("q", 4),
                          ("q", 10): ("q", 0),
                          ("q", 11): ("q", 13),
                          ("q", 12): ("q", 1),
                          ("q", 13): ("q", 12)
                  #[3, 2, 10, 11, 5, 9, 8, 6, 7, 4, 0, 13, 1, 12]
                         } 
    elif(workload == 2):
        initial_layout = {("q", 0): ("q", 1),
                          ("q", 1): ("q", 2),
                          ("q", 2): ("q", 3),
                          ("q", 3): ("q", 4),
                          ("q", 4): ("q", 5),
                          ("q", 5): ("q", 6),
                          ("q", 6): ("q", 8),
                          ("q", 7): ("q", 9),
                          ("q", 8): ("q", 10),
                          ("q", 9): ("q", 13),
                          ("q", 10): ("q", 11),
                          ("q", 11): ("q", 0),
                          ("q", 12): ("q", 12),
                          ("q", 13): ("q", 7)}
        # Execute noisy simulation and get counts
    elif(workload == 3):
        initial_layout = {("q", 0): ("q", 10),
                          ("q", 1): ("q", 3),
                          ("q", 2): ("q", 11),
                          ("q", 3): ("q", 12),
                          ("q", 4): ("q", 0),
                          ("q", 5): ("q", 6),
                          ("q", 6): ("q", 4),
                          ("q", 7): ("q", 7),
                          ("q", 8): ("q", 9),
                          ("q", 9): ("q", 8),
                          ("q", 10): ("q", 2),
                          ("q", 11): ("q", 13),
                          ("q", 12): ("q", 5),
                          ("q", 13): ("q", 1)}
    elif(workload == 4):       
        initial_layout = {("q", 0): ("q", 2),
                          ("q", 1): ("q", 1),
                          ("q", 2): ("q", 12),
                          ("q", 3): ("q", 13),
                          ("q", 4): ("q", 9),
                          ("q", 5): ("q", 0),
                          ("q", 6): ("q", 7),
                          ("q", 7): ("q", 10),
                          ("q", 8): ("q", 8),
                          ("q", 9): ("q", 5),
                          ("q", 10): ("q", 3),
                          ("q", 11): ("q", 6),
                          ("q", 12): ("q", 11),
                          ("q", 13): ("q", 4)}
    elif(workload == 5):
        initial_layout = {("q", 0): ("q", 1),
                          ("q", 1): ("q", 2),
                          ("q", 2): ("q", 10),
                          ("q", 3): ("q", 3),
                          ("q", 4): ("q", 9),
                          ("q", 5): ("q", 8),
                          ("q", 6): ("q", 0),
                          ("q", 7): ("q", 13),
                          ("q", 8): ("q", 11),
                          ("q", 9): ("q", 5),
                          ("q", 10): ("q", 6),
                          ("q", 11): ("q", 7),
                          ("q", 12): ("q", 12),
                          ("q", 13): ("q", 4)}
        
        
    result_noise = execute(circ, simulator, shots = 8192,
                       noise_model=noise_model,
                       coupling_map=coupling_map,
                       initial_layout = initial_layout,
                       basis_gates=basis_gates).result()
    counts_noise = result_noise.get_counts(circ)
    # Execute on ideal simulator
    if(workload==1):
        counts_ideal = ideal_simulation(bv9)
    elif(workload==2):
        counts_ideal = ideal_simulation(ghz8)
    elif(workload==3):
        counts_ideal = ideal_simulation(fredkin) 
    elif(workload==4):
        counts_ideal = ideal_simulation(decode)     
    elif(workload==5):
        counts_ideal = ideal_simulation(qaoa)  
    pst = get_PST(counts_noise, counts_ideal)
    print('Ideal Counts ', counts_ideal)
    print('Noise Counts ', counts_noise)
    print('PST ', pst)
    #plot_histogram(counts_noise, title="Counts for 3-qubit GHZ state with depolarizing noise model")

In [ ]:
run_simulation(5)